In [1]:
import pandas as pd
import numpy as np
from math import sqrt
from statsmodels.stats.power import TTestPower

In [2]:
df = pd.read_excel("../Fig2|EDFig1b_2_3|TableS2/ST1_oralASV_mouse.xlsx", sheet_name="Table S1b")
df.head()

,Sample ID,Collection body site,Collection time point,Mouse ID,Treatment group,Exclusion,Total bacterial load,Oral bacterial fraction,Oral bacterial load,Gut bacterial fraction,Gut bacterial load
0,KP_Exp_8_D0_AVN1,Fecal,pre,Abx_1A,Antibiotic,No,4.195976e+11,0.000035,1.482712e+07,0.999965,4.195827e+11
1,KP_Exp_8_D8_AVN1,Fecal,w1,Abx_1A,Antibiotic,No,1.743481e+08,0.224412,3.912579e+07,0.775588,1.352224e+08
2,KP_Exp_8_D0_OR_AVN1,Oral,pre,Abx_1A,Antibiotic,No,NaN,NaN,NaN,NaN,NaN
3,KP_Exp_8_D0_AVN2,Fecal,pre,Abx_1B,Antibiotic,No,4.947265e+11,0.000043,2.107843e+07,0.999957,4.947054e+11
4,KP_Exp_8_D8_AVN2,Fecal,w1,Abx_1B,Antibiotic,No,1.466698e+08,0.289092,4.240105e+07,0.710908,1.042687e+08


# First experiment only

In [3]:
df1 = df[df['Mouse ID'].str.contains('Abx_1')][['Mouse ID','Collection time point','Gut bacterial load']]
df1 = pd.pivot_table(df1, index='Mouse ID', columns='Collection time point', values='Gut bacterial load')
df1

Collection time point,pre,w1
Mouse ID,,
Abx_1A,4.195827e+11,1.352224e+08
Abx_1B,4.947054e+11,1.042687e+08
Abx_1C,5.361720e+11,9.092405e+07


In [4]:
n_pre = 3
n_post = 3
gutload_pre = list(df1.pre)
gutload_post = list(df1.w1)
s_pre, s_post = np.var(gutload_pre), np.var(gutload_post)
s = sqrt(((n_pre - 1) * s_pre + (n_post - 1) * s_post) / (n_pre + n_post - 2)) # calculate the pooled standard deviation 
u_pre, u_post = np.mean(gutload_pre), np.mean(gutload_post) # means of the samples
d = (u_pre - u_post) / s # calculate the effect size
alpha = 0.05 # significance leve
obj = TTestPower()
power = obj.power(effect_size=d, nobs=3, alpha=alpha, df=None, alternative='larger')
print('power of 3 mice: %.6f'%(power))

power of 3 mice: 1.000000


# Combine both experiments

## Mouse Abx_2B lacks pre-treatment fecal sample

In [5]:
df12 = df[df['Mouse ID'].str.contains('Abx')][['Mouse ID','Collection time point','Gut bacterial load']]
df12 = pd.pivot_table(df12, index='Mouse ID', columns='Collection time point', values='Gut bacterial load')
df12['post'] = np.nanmean(df12[['d3','w1']], axis=1)
df12 = df12[['pre','post']]
df12 = df12[(df12.pre.notnull()) & (df12.post.notnull())]
df12

Collection time point,pre,post
Mouse ID,,
Abx_1A,4.195827e+11,1.352224e+08
Abx_1B,4.947054e+11,1.042687e+08
Abx_1C,5.361720e+11,9.092405e+07
Abx_2A,1.153214e+10,1.255068e+07
Abx_2C,1.222237e+10,2.140783e+06
Abx_2D,8.795456e+09,4.037800e+06
Abx_2E,9.157845e+09,3.101776e+06


In [6]:
n_pre = 7
n_post = 7
gutload_pre = list(df12.pre)
gutload_post = list(df12.post)
s_pre, s_post = np.var(gutload_pre), np.var(gutload_post)
s = sqrt(((n_pre - 1) * s_pre + (n_post - 1) * s_post) / (n_pre + n_post - 2)) # calculate the pooled standard deviation 
u_pre, u_post = np.mean(gutload_pre), np.mean(gutload_post) # means of the samples
d = (u_pre - u_post) / s # calculate the effect size
alpha = 0.05 # significance leve
obj = TTestPower()
obj = TTestPower()
power = obj.power(effect_size=d, nobs=7, alpha=alpha, df=None, alternative='larger')
print('power of 7 mice: %.6f'%(power))

power of 7 mice: 0.906716
